In [ ]:
!pip install ipython-autotime
%load_ext autotime
!pip install spotipy
import spotipy
import pandas as pd
import re
# tqdm mostra barra de processo https://github.com/tqdm/tqdm
from tqdm import tqdm
from spotipy.oauth2 import SpotifyClientCredentials

# Usuário: fijita8647@karavic.com
# Senha: INF10322803
cid ="" 
secret = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

     |████████████████████████████████| 138 kB 7.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
time: 6.65 s (started: 2022-04-03 21:14:46 +00:00)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

time: 21.5 s (started: 2022-04-03 21:17:17 +00:00)


In [2]:
# csv retirado de https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge
path = "/content/drive/MyDrive/INF1032 - Spotify/100000-100999.csv"
# arquivo original tem +60k linhas, então pegamos as primeiras 1k linhas pela demora
# em requisitar os dados das músicas para a api do spotify
# para 1000 linhas demorou por volta de 11min
csv_df = pd.read_csv(path, nrows=1000)
csv_df.head()

FileNotFoundError: ignored

time: 231 ms (started: 2022-04-03 21:14:53 +00:00)


In [ ]:
# baseado no código https://github.com/enjuichang/PracticalDataScience-ENCA/blob/main/notebooks/scripts/ari.py
def requisicao_api_spotify(uri):
    #Audio features: https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features
    features = sp.audio_features(uri)[0]
    
    #Artist of the track, for genres and popularity
    # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track
    # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artist
    # pega o artista principal da música e não leva em consideração feats
    artist = sp.track(uri)["artists"][0]["id"]
    artist_pop = sp.artist(artist)["popularity"]
    artist_genres = sp.artist(artist)["genres"]

    #Track popularity
    track_pop = sp.track(uri)["popularity"]
    
    #Add in extra features
    features["artist_pop"] = artist_pop
    features["artist_followers"] = sp.artist(artist)["followers"]["total"]
    features["release_date"] = sp.track(uri)["album"]["release_date"]
    if artist_genres:
        features["genres"] = ",".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features["genres"] = "unknown"
    features["track_pop"] = track_pop
    
    return features

In [ ]:
ids_list = csv_df["track_uri"]
features_list = []

for i in tqdm([uri for uri in ids_list]):
    try:
        features_list.append(requisicao_api_spotify(i))
    except:
        continue

features_df = pd.DataFrame(features_list)
features_df.head()

In [ ]:
musicas_df = pd.concat([csv_df,features_df], axis=1)
musicas_df.head()